Trainingsdatenset einlesen

In [14]:
import pandas as pd
training_set = pd.read_csv('data/train.csv')


Trainingsset aufteilen in 80% training und 20% validierung

In [15]:
from sklearn.model_selection import train_test_split
train_set,validation_set = train_test_split(training_set, test_size=0.2)

Regel erstellen für das Überleben eines Passagiers aufgrund der Klasse.

In [16]:
class_all = {1:0, 2:0, 3:0}
class_survived = {1:0, 2:0, 3:0}

for entry in train_set.iterrows():
    class_all[entry[1]['Pclass']] += 1
    class_survived[entry[1]['Pclass']] += entry[1]['Survived']

# print out the survival rate for each class
class_survival_rate = {1:0, 2:0, 3:0}
for i in range(1,4):
    class_survival_rate[i] = class_survived[i] / class_all[i]
    print('Class ' + str(i) + ': ' + str(class_survival_rate[i]))

print(class_survival_rate)



Class 1: 0.6411764705882353
Class 2: 0.49333333333333335
Class 3: 0.23214285714285715
{1: 0.6411764705882353, 2: 0.49333333333333335, 3: 0.23214285714285715}


Regel auf Validierungsset anwenden und Accuracy berechnen

In [17]:
import random

pred_rate = {'right': 0, 'wrong': 0}
print(len(validation_set))
print(len(train_set))

for entry in validation_set.iterrows():
    # get class of passenger and generate prediciton based on survival rate
    pclass = class_survival_rate[entry[1]['Pclass']]

    prediction = (random.random() < pclass)
    
    # compare prediction with actual survival
    if prediction == entry[1]['Survived']:
        pred_rate['right'] += 1
    else:
        pred_rate['wrong'] += 1

print(pred_rate)
print(pred_rate['right'] / (pred_rate['right'] + pred_rate['wrong']))
    



179
712
{'right': 112, 'wrong': 67}
0.6256983240223464


import test datenset und wende Regel auf jeden Passenger an

In [18]:
test_set = pd.read_csv('data/test.csv')

# Make predictions for each passenger in the test set
predictions = []
for idx, entry in test_set.iterrows():
    pclass = class_survival_rate[entry['Pclass']]
    prediction = (random.random() < pclass)
    predictions.append(int(prediction))

# Create the output DataFrame with passenger IDs and predictions
output = pd.DataFrame({'PassengerId': test_set['PassengerId'], 'Survived': predictions})

# Write the output DataFrame to a CSV file
output.to_csv('submission.csv', index=False)


Logistische Regression mit Pclass

In [19]:
import sklearn.linear_model as lm

# validation_set['Age'].fillna(train_set['Age'].mean(), inplace=True)
# train_set["Age"].fillna(train_set["Age"].median(skipna=True), inplace=True)

# Create a new logistic regression model
model = lm.LogisticRegression()

# Train the model using pClass and Sex
model.fit(train_set[['Pclass']], train_set[['Survived']])

# Calculate the accuracy on the validation se

print(model.score(validation_set[['Pclass']], validation_set['Survived']))

0.664804469273743


C:\Users\Lars\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Use get_dummies for better results

In [20]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import math

train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")

x, x_tt = train_test_split(train_data, test_size=0.2) #Split data into a trainining set = x and a test training set = x_tt to calculate accuracy later

model = LogisticRegression() #Setup the model

y      = x["Survived"]      #specify the y-value to train the model
y_tt   = x_tt["Survived"]   #do the same for the training test data set

features = ["Pclass"] #specify the features using a list to play with the parameters later
x      = pd.get_dummies(x[features]) #define the training input data
x_tt   = pd.get_dummies(x_tt[features])  #defines the training test data to calculate accuracy

x_test = pd.get_dummies(test_data[features]) #define the real test data input

model.fit(x, y) #train the model with the training data

predictions_train = model.predict(x_tt) #prediction using the test training data set

predictions       = model.predict(x_test) #prediction using the real test data provided by kaggle

acc = accuracy_score(y_tt, predictions_train)
print("Accuracy: ~", math.ceil(acc * 100), "%")

Accuracy: ~ 68 %


Add age as feature

In [21]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import math

train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")

train_data["Age"].fillna(train_data["Age"].median(skipna=True), inplace=True) #Fill missing age in train data
test_data["Age"].fillna(test_data["Age"].median(skipna=True), inplace=True)   #Fill missing age in test data

x, x_tt = train_test_split(train_data, test_size=0.2) #Split data into a trainining set = x and a test training set = x_tt to calculate accuracy later

model = LogisticRegression() #Setup the model

y      = x["Survived"]      #specify the y-value to train the model
y_tt   = x_tt["Survived"]   #do the same for the training test data set

features = ["Pclass", "Age"] #specify the features using a list to play with the parameters later
x      = pd.get_dummies(x[features]) #define the training input data
x_tt   = pd.get_dummies(x_tt[features])  #defines the training test data to calculate accuracy

x_test = pd.get_dummies(test_data[features]) #define the real test data input

model.fit(x, y) #train the model with the training data

predictions_train = model.predict(x_tt) #prediction using the test training data set

predictions       = model.predict(x_test) #prediction using the real test data provided by kaggle

acc = accuracy_score(y_tt, predictions_train)
print("Accuracy: ~", math.ceil(acc * 100), "%")

Accuracy: ~ 68 %


Use RandomForestClassifier as model

In [30]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import math
from sklearn.ensemble import RandomForestClassifier

train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")

train_data["Age"].fillna(train_data["Age"].median(skipna=True), inplace=True) #Fill missing age in train data
test_data["Age"].fillna(test_data["Age"].median(skipna=True), inplace=True)   #Fill missing age in test data

x, x_tt = train_test_split(train_data, test_size=0.2) #Split data into a trainining set = x and a test training set = x_tt to calculate accuracy later

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1) #Setup the model

y      = x["Survived"]      #specify the y-value to train the model
y_tt   = x_tt["Survived"]   #do the same for the training test data set

features = ["Pclass", "Age", "Sex", "SibSp", "Parch"] #specify the features using a list to play with the parameters later
x      = pd.get_dummies(x[features]) #define the training input data
x_tt   = pd.get_dummies(x_tt[features])  #defines the training test data to calculate accuracy

x_test = pd.get_dummies(test_data[features]) #define the real test data input

model.fit(x, y) #train the model with the training data

predictions_train = model.predict(x_tt) #prediction using the test training data set

predictions       = model.predict(x_test) #prediction using the real test data provided by kaggle

acc = accuracy_score(y_tt, predictions_train)
print("Accuracy: ~", math.ceil(acc * 100), "%")

Accuracy: ~ 84 %


Submit to Kaggle

In [32]:
output = pd.DataFrame({'PassengerId': test_set['PassengerId'], 'Survived': predictions})

# Write the output DataFrame to a CSV file
output.to_csv('submission_new.csv', index=False)